# 01: Shenzhen Synthetic Data Preparation

From: jodafons 

This notebook will be responsible to create a final spreadsheet with all images paths, hashes and clinical readings.

In [1]:
import pandas as pd
import numpy as np
import os, sys
import glob
import re
import hashlib
import pathlib
from tqdm import tqdm
#from sklearn.model_selection import train_test_split
from rxwgan.core import stratified_train_val_test_splits
import warnings
warnings.filterwarnings("ignore")


2022-12-12 20:18:51.312979: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-12 20:18:51.514219: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-12 20:18:54.114316: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: UNKNOWN ERROR (34)
2022-12-12 20:18:54.114404: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (caloba52): /proc/driver/nvidia/version does not e

In [2]:
def expand_folder( path , extension):
    l = glob.glob(path+'/*.'+extension)
    l.sort()
    return l

In [3]:
def get_md5(path):
    return hashlib.md5(pathlib.Path(path).read_bytes()).hexdigest()

In [4]:
def get_template():
    # template
    return  {
      'dataset_name': [],
      'dataset_type': [],
      'project_id': [],
      'target': [],
      #'image_md5':[],
      #'image_url': [],
      'raw_image_path':[],
      'insertion_date': [],
      'metadata': [],
      'date_acquisition': [],
      'test':[],
      'sort':[],
      'type':[],
      }

## Create wgan table:

In [9]:
basepath = '/home/joao.pinto/public/brics_data/Shenzhen/fake_images/wgan/v2'


def create_table(path, target, dataset_name, date_acquisition):

    sorts=9
    tests = 10
    d = get_template()
    
    bins = []
    for test in range(tests):
        for sort in range(sorts):
            bins.append((test,sort))
    
    for test, sort in tqdm(bins):
            _path = path+'/job.test_%d.sort_%d'%(test,sort)
            l = glob.glob(_path+'/*.png')
            l.sort()
            for image_path in l:
                filename = image_path.split('/')[-1]
                d['target'].append(target)
                d['raw_image_path'].append(image_path)
                d['project_id'].append(filename.replace('.png',''))
                d['dataset_name'].append(dataset_name)
                d['dataset_type'].append('fake')
                d['sort'].append(sort)
                d['test'].append(test)
                d['type'].append('test')
                d['metadata'].append({})
                d['insertion_date'].append('')
                d['date_acquisition'].append(date_acquisition)
                
    return pd.DataFrame(d)


d1 = create_table( basepath+'/user.jodafons.images.Shenzhen.wgan.3k_samples.v2_notb.r1', 0, 'Shenzhen_wgan_v2', '2022-06-22')
d2 = create_table( basepath+'/user.jodafons.images.Shenzhen.wgan.3k_samples.v2_tb.r1', 1, 'Shenzhen_wgan_v2', '2022-06-22')

df_fake = pd.concat((d1,d2))
df_fake.to_csv('user.joao.pinto.images.Shenzhen.wgan.3k_samples.v2_new.csv')

100%|██████████████████████████████████████████████| 90/90 [00:15<00:00,  5.98it/s]


In [10]:
df_fake

,dataset_name,dataset_type,project_id,target,raw_image_path,insertion_date,metadata,date_acquisition,test,sort,type
0,Shenzhen_wgan_v2,fake,fake.notb.test_0.sort_0.000000,0,/home/joao.pinto/public/brics_data/Shenzhen/fa...,,{},2022-06-22,0,0,test
1,Shenzhen_wgan_v2,fake,fake.notb.test_0.sort_0.000001,0,/home/joao.pinto/public/brics_data/Shenzhen/fa...,,{},2022-06-22,0,0,test
2,Shenzhen_wgan_v2,fake,fake.notb.test_0.sort_0.000002,0,/home/joao.pinto/public/brics_data/Shenzhen/fa...,,{},2022-06-22,0,0,test
3,Shenzhen_wgan_v2,fake,fake.notb.test_0.sort_0.000003,0,/home/joao.pinto/public/brics_data/Shenzhen/fa...,,{},2022-06-22,0,0,test
4,Shenzhen_wgan_v2,fake,fake.notb.test_0.sort_0.000004,0,/home/joao.pinto/public/brics_data/Shenzhen/fa...,,{},2022-06-22,0,0,test
...,...,...,...,...,...,...,...,...,...,...,...
287995,Shenzhen_wgan_v2,fake,fake.tb.test_9.sort_8.003195,1,/home/joao.pinto/public/brics_data/Shenzhen/fa...,,{},2022-06-22,9,8,test
287996,Shenzhen_wgan_v2,fake,fake.tb.test_9.sort_8.003196,1,/home/joao.pinto/public/brics_data/Shenzhen/fa...,,{},2022-06-22,9,8,test
287997,Shenzhen_wgan_v2,fake,fake.tb.test_9.sort_8.003197,1,/home/joao.pinto/public/brics_data/Shenzhen/fa...,,{},2022-06-22,9,8,test
287998,Shenzhen_wgan_v2,fake,fake.tb.test_9.sort_8.003198,1,/home/joao.pinto/public/brics_data/Shenzhen/fa...,,{},2022-06-22,9,8,test


## Create P2P table:

In [11]:
basepath = '/home/joao.pinto/public/brics_data/Shenzhen/fake_images/pix2pix/v1/'


def create_table(path, target, dataset_name, date_acquisition):

    sorts=9
    tests = 10
    d = get_template()
    
    bins = []
    for test in range(tests):
        for sort in range(sorts):
            bins.append((test,sort))
    
    for test, sort in tqdm(bins):
        
            for tname in ['TRAIN','VAL','TEST']:

                _path = path+'/job.test_%d.sort_%d/p2p_%s/%s'%(test,sort, 'NTB' if target==0 else 'TB', tname)
                l = glob.glob(_path+'/*.png')
                l.sort()
                for image_path in l:
                    filename = image_path.split('/')[-1]
                    d['target'].append(target) 
                    
                    if 'real_A.png' in image_path:
                        d['type'].append('mask')
                        d['project_id'].append(filename.replace('_real_A.png',''))
                    elif 'real_B.png' in image_path:
                        d['type'].append('real')
                        d['project_id'].append(filename.replace('_real_B.png',''))
                    else:
                        d['type'].append('fake')
                        d['project_id'].append(filename.replace('_fake_B.png',''))
                        
                    d['raw_image_path'].append(image_path)
                    d['dataset_name'].append(dataset_name)
                    d['dataset_type'].append('fake')
                    d['sort'].append(sort)
                    d['test'].append(test)
                    d['metadata'].append({})
                    d['insertion_date'].append('')
                    d['date_acquisition'].append(date_acquisition)
                
                
    return pd.DataFrame(d)


d1 = create_table( basepath+'/user.otavares.Shenzhen.pix2pix.v1_notb.r1', 0, 'Shenzhen_pix2pix_v1','2022-06-22')
d2 = create_table( basepath+'/user.otavares.Shenzhen.pix2pix.v1_tb.r1', 1, 'Shenzhen_pix2pix_v1','2022-06-22')
df_fake = pd.concat((d1,d2),axis=0)
df_fake.to_csv('user.otto.tavares.Shenzhen.pix2pix.v1_tb.r1_new.csv')

100%|██████████████████████████████████████████████| 90/90 [00:04<00:00, 20.56it/s]


In [12]:
df_fake

,dataset_name,dataset_type,project_id,target,raw_image_path,insertion_date,metadata,date_acquisition,test,sort,type
0,Shenzhen_pix2pix_v1,fake,CHNCXR_0001_0_0,0,/home/joao.pinto/public/brics_data/Shenzhen/fa...,,{},2022-06-22,0,0,fake
1,Shenzhen_pix2pix_v1,fake,CHNCXR_0001_0_0,0,/home/joao.pinto/public/brics_data/Shenzhen/fa...,,{},2022-06-22,0,0,mask
2,Shenzhen_pix2pix_v1,fake,CHNCXR_0001_0_0,0,/home/joao.pinto/public/brics_data/Shenzhen/fa...,,{},2022-06-22,0,0,real
3,Shenzhen_pix2pix_v1,fake,CHNCXR_0002_0_0,0,/home/joao.pinto/public/brics_data/Shenzhen/fa...,,{},2022-06-22,0,0,fake
4,Shenzhen_pix2pix_v1,fake,CHNCXR_0002_0_0,0,/home/joao.pinto/public/brics_data/Shenzhen/fa...,,{},2022-06-22,0,0,mask
...,...,...,...,...,...,...,...,...,...,...,...
77485,Shenzhen_pix2pix_v1,fake,CHNCXR_0639_1_0,1,/home/joao.pinto/public/brics_data/Shenzhen/fa...,,{},2022-06-22,9,8,mask
77486,Shenzhen_pix2pix_v1,fake,CHNCXR_0639_1_0,1,/home/joao.pinto/public/brics_data/Shenzhen/fa...,,{},2022-06-22,9,8,real
77487,Shenzhen_pix2pix_v1,fake,CHNCXR_0646_1_0,1,/home/joao.pinto/public/brics_data/Shenzhen/fa...,,{},2022-06-22,9,8,fake
77488,Shenzhen_pix2pix_v1,fake,CHNCXR_0646_1_0,1,/home/joao.pinto/public/brics_data/Shenzhen/fa...,,{},2022-06-22,9,8,mask
